In [1]:
# !pip install --index-url https://test.pypi.org/simple/ gc2-python-client

In [2]:
import gc2
import folium
import pandas

In [3]:
# local = gc2.Gc2("https://dk.gc2.io")
local = gc2.Gc2("http://127.0.0.1:8080")
# local.set_authentication(user="dk_test", pw="Pwd1234!", db="dk")
local.set_authentication(user="mydb", pw="hawk2000", db="mydb")

In [4]:
schema = gc2.Schema(local)
schema.get()
pandas.DataFrame(schema.data)

,schema,count
0,_gc2scheduler,14814
1,_system,0
2,adsdd,0
3,dar,1
4,dokumenter,1
5,fot,56
6,gasledninger,4
7,geodk,5
8,geometries,7
9,grids,13


In [5]:
meta = gc2.Meta(local)
meta.get("test.multipolygon")
pandas.DataFrame(meta.data)

,test.multipolygon.the_geom
authentication,None
children,None
coord_dimension,2
f_geometry_column,the_geom
f_table_abstract,None
f_table_name,multipolygon
f_table_schema,test
f_table_title,None
fields,"{'gid': {'num': 1, 'type': 'integer', 'full_ty..."
layergroup,Pane


In [6]:
query = gc2.Sql(local)
query.run("select ejerlavsnavn, matrikelnummer, 1 as hej, the_geom from matrikel.jordstykke limit 100")
pandas.DataFrame(query.data, columns=query.columns)

,ejerlavsnavn,matrikelnummer,hej
0,Skanderborg Bygrunde,87b,1
1,Skanderborg Bygrunde,88a,1
2,Skanderborg Bygrunde,88c,1
3,Skanderborg Bygrunde,7000a,1
4,Skanderborg Bygrunde,87a,1
...,...,...,...
95,Skanderborg Bygrunde,78e,1
96,Skanderborg Bygrunde,75a,1
97,Skanderborg Bygrunde,190b,1
98,Skanderborg Bygrunde,191a,1


In [7]:
m = folium.Map(
    location=[56.1759, 10.6016],
    tiles="cartodbpositron",
    zoom_start=7,
)
folium.GeoJson(query.geojson, name="geojson").add_to(m)
folium.LayerControl().add_to(m)
m